# Household Structured Epidemic Model

Here we use a transmission rule that allows transmission to differ based on whether people have some shared trait to create an epidemic model with household structure. Here folks are far more likely to transmit to others in their household, and move through the standard S, I and R states once infected. 

First imports and setup.

In [1]:
import tabularepimdl as tepi

import pandas as pd
import numpy as np

## Defining a Population

The population is going to consist of 500 households with a mean size of 2. We will add an infected person to the last household on the list (HH number 499). 

In [3]:
##create a population with 500 households of average size 2 and one
##person infected in the 500th household
hh_pop = pd.DataFrame({
    'DisState':pd.Categorical(["S"]*500+["I"],  categories=['S','I','R']),
    'HH': list(range(500))+['499'],
    'N': list(1+np.random.poisson(1,500))+[1],
    'T':0
})

hh_pop.tail()

,DisState,HH,N,T
496,S,496,3,0
497,S,497,1,0
498,S,498,2,0
499,S,499,3,0
500,I,499,1,0


## Creating an Epidemic Model

To define the epidemic model we will use a transmission rule that allows us to specify different transmission probabilities for those inside and outside of the household. Here we will assume that those inside the household have a 20% chance of transmitting to a fellow household member, and that they have a 0.2% chance transmitting to someone outside. Given an average household size of 0.2 and a total population of approximately 1000, $R_0$ is approximately:
$$R_0 \approx (0.2)*(2) + (0.02)(1000) = 2.4$$

We will further presume infection lasts an average of 5 days.

In [4]:
hhEpi = tepi.EpiModel(hh_pop,
                 [tepi.SharedTraitInfection(.2/5,.002/5,"DisState","HH"),
                  tepi.SimpleTransition("DisState","I","R",.02)])

## Comparison with an unstructured SIR model

How does houehold structure change the shape of the epidemic curve, even if transmission is determinstic? We will need a standard SIR model with an equivalent R0 to compare to.

In [14]:
#First let's get the an equivalent population that does not have the population structure.
simp_pop = hh_pop.copy().drop("HH", axis=1)
simp_pop = simp_pop.groupby(["DisState","T"]).sum().reset_index()


# Now create the standard SIR epidemic model


,DisState,T,N
0,S,0,1005
1,I,0,1
2,R,0,0


## Comparing Stochastic and Deterministic Results

The household process is actually a bit more intesting from the stochastic prospective. Let's now compare that with the deterministic results, particularly around differences in household attack rates.
